In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pickle, tempfile
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_svmlight_file
from sklearn import svm
from nltk.parse import DependencyGraph, DependencyEvaluator
from nltk.parse.transitionparser import TransitionParser, Configuration, Transition
from os import remove
from copy import deepcopy
from operator import itemgetter
from numpy import array
from scipy import sparse

In [0]:
#pre-processing data
def load_data(filename):
    tg = []
    with open(filename,'r') as f:
        con= f.read().split('\n\n')
#         print("Number of sentences :",len(con),end='\n\n')
        for s in con:
            t = s.split('\n')
            s = [i for i in t if not i.startswith('#')]
            tg.append(DependencyGraph(s))
    return tg

In [4]:
train_graph_eng = load_data('/content/drive/My Drive/NLP/Corpus/eng.train.conllu')
test_graph_eng = load_data('/content/drive/My Drive/NLP/Corpus/eng.test.conllu')
train_graph_sp = load_data('/content/drive/My Drive/NLP/Corpus/spanish.train.conllu')
test_graph_sp = load_data('/content/drive/My Drive/NLP/Corpus/spanish.test.conllu')

/usr/local/lib/python3.6/dist-packages/nltk/parse/dependencygraph.py:380: UserWarning: The graph doesn't contain a node that depends on the root element.
  "The graph doesn't contain a node "


In [0]:
class modifiedConfiguration(Configuration):
    def extract_features(self, include_features):
        """
        Extract the set of features for the current configuration. Implement standard features as describe in
        Table 3.2 (page 31) in Dependency Parsing book by Sandra Kubler, Ryan McDonal, Joakim Nivre.
        Please note that these features are very basic.
        :return: list(str)
        """
        result = []
        # Todo : can come up with more complicated features set for better
        # performance.
        if len(self.stack) > 0:
            # Stack 0
            stack_idx0 = self.stack[len(self.stack) - 1]
            token = self._tokens[stack_idx0]
            if self._check_informative(token['word'], True):
                if include_features['word']:
                    result.append('STK_0_FORM_' + token['word'])
                else:
                    result.append('STK_0_FORM_' + '#')
            if 'lemma' in token and self._check_informative(token['lemma']):
                if include_features['lemma']:
                    result.append('STK_0_LEMMA_' + token['lemma'])
                else:
                    result.append('STK_0_LEMMA_' + '#')
            if self._check_informative(token['tag']):
                if include_features['tag']:
                    result.append('STK_0_POS_' + token['tag'])
                else:
                    result.append('STK_0_POS_' + '#')
            if 'feats' in token and self._check_informative(token['feats']):
                feats = token['feats'].split("|")
                if include_features['feats']:
                    for feat in feats:
                        result.append('STK_0_FEATS_' + feat)
                else:
                    result.append('STK_0_FEATS_' + '#')
                    
            # Stack 1
            if len(self.stack) > 1:
                stack_idx1 = self.stack[len(self.stack) - 2]
                token = self._tokens[stack_idx1]
                if self._check_informative(token['tag']):
                    if include_features['tag']:
                        result.append('STK_1_POS_' + token['tag'])
                    else:
                        result.append('STK_1_POS_' + '#')

            # Left most, right most dependency of stack[0]
            left_most = 1000000
            right_most = -1
            dep_left_most = ''
            dep_right_most = ''
            for (wi, r, wj) in self.arcs:
                if wi == stack_idx0:
                    if (wj > wi) and (wj > right_most):
                        right_most = wj
                        dep_right_most = r
                    if (wj < wi) and (wj < left_most):
                        left_most = wj
                        dep_left_most = r
            if self._check_informative(dep_left_most):
                result.append('STK_0_LDEP_' + dep_left_most)
            if self._check_informative(dep_right_most):
                result.append('STK_0_RDEP_' + dep_right_most)

        # Check Buffered 0
        if len(self.buffer) > 0:
            # Buffer 0
            buffer_idx0 = self.buffer[0]
            token = self._tokens[buffer_idx0]
            if self._check_informative(token['word'], True):
                if include_features['word']:
                    result.append('BUF_0_FORM_' + token['word'])
                else:
                    result.append('BUF_0_FORM_' + '#')
            if 'lemma' in token and self._check_informative(token['lemma']):
                if include_features['lemma']:
                    result.append('BUF_0_LEMMA_' + token['lemma'])
                else:
                    result.append('BUF_0_LEMMA_' + '#')
            if self._check_informative(token['tag']):
                if include_features['tag']:
                    result.append('BUF_0_POS_' + token['tag'])
                else:
                    result.append('BUF_0_POS_' + '#')
            if 'feats' in token and self._check_informative(token['feats']):
                feats = token['feats'].split("|")
                if include_features['feats']:
                    for feat in feats:
                        result.append('BUF_0_FEATS_' + feat)
                else:
                    result.append('BUF_0_FEATS_' + '#')
            # Buffer 1
            if len(self.buffer) > 1:
                buffer_idx1 = self.buffer[1]
                token = self._tokens[buffer_idx1]
                if self._check_informative(token['word'], True):
                    if include_features['word']:
                        result.append('BUF_1_FORM_' + token['word'])
                    else:
                        result.append('BUF_1_FORM_' + '#')
                if self._check_informative(token['tag']):
                    if include_features['tag']:
                        result.append('BUF_1_POS_' + token['tag'])
                    else:
                        result.append('BUF_1_POS_' + '#')
            if len(self.buffer) > 2:
                buffer_idx2 = self.buffer[2]
                token = self._tokens[buffer_idx2]
                if self._check_informative(token['tag']):
                    if include_features['tag']:
                        result.append('BUF_2_POS_' + token['tag'])
                    else:
                        result.append('BUF_2_POS_' + '#')
            if len(self.buffer) > 3:
                buffer_idx3 = self.buffer[3]
                token = self._tokens[buffer_idx3]
                if self._check_informative(token['tag']):
                    if include_features['tag']:
                        result.append('BUF_3_POS_' + token['tag'])
                    else:
                        result.append('BUF_3_POS_' + '#')
                    # Left most, right most dependency of stack[0]
            left_most = 1000000
            right_most = -1
            dep_left_most = ''
            dep_right_most = ''
            for (wi, r, wj) in self.arcs:
                if wi == buffer_idx0:
                    if (wj > wi) and (wj > right_most):
                        right_most = wj
                        dep_right_most = r
                    if (wj < wi) and (wj < left_most):
                        left_most = wj
                        dep_left_most = r
            if self._check_informative(dep_left_most):
                result.append('BUF_0_LDEP_' + dep_left_most)
            if self._check_informative(dep_right_most):
                result.append('BUF_0_RDEP_' + dep_right_most)

        return result

In [0]:
class modifiedTransitionParser(TransitionParser):
    def _create_training_examples_arc_std(self, depgraphs, input_file, include_features):
        operation = Transition(self.ARC_STANDARD)
        count_proj = 0
        training_seq = []

        for depgraph in depgraphs:
            if not self._is_projective(depgraph):
                continue

            count_proj += 1
            conf = modifiedConfiguration(depgraph)
            while len(conf.buffer) > 0:
                b0 = conf.buffer[0]
                features = conf.extract_features(include_features)
                binary_features = self._convert_to_binary_features(features)

                if len(conf.stack) > 0:
                    s0 = conf.stack[len(conf.stack) - 1]
                    # Left-arc operation
                    rel = self._get_dep_relation(b0, s0, depgraph)
                    if rel is not None:
                        key = Transition.LEFT_ARC + ':' + rel
                        self._write_to_file(key, binary_features, input_file)
                        operation.left_arc(conf, rel)
                        training_seq.append(key)
                        continue

                    # Right-arc operation
                    rel = self._get_dep_relation(s0, b0, depgraph)
                    if rel is not None:
                        precondition = True
                        # Get the max-index of buffer
                        maxID = conf._max_address

                        for w in range(maxID + 1):
                            if w != b0:
                                relw = self._get_dep_relation(b0, w, depgraph)
                                if relw is not None:
                                    if (b0, relw, w) not in conf.arcs:
                                        precondition = False

                        if precondition:
                            key = Transition.RIGHT_ARC + ':' + rel
                            self._write_to_file(
                                key,
                                binary_features,
                                input_file)
                            operation.right_arc(conf, rel)
                            training_seq.append(key)
                            continue

                # Shift operation as the default
                key = Transition.SHIFT
                self._write_to_file(key, binary_features, input_file)
                operation.shift(conf)
                training_seq.append(key)

        print(" Number of training examples : " + str(len(depgraphs)))
        print(" Number of valid (projective) examples : " + str(count_proj))
        return training_seq
    
    def _create_training_examples_arc_eager(self, depgraphs, input_file, include_features):
        """
        Create the training example in the libsvm format and write it to the input_file.
        Reference : 'A Dynamic Oracle for Arc-Eager Dependency Parsing' by Joav Goldberg and Joakim Nivre
        """
        operation = Transition(self.ARC_EAGER)
        countProj = 0
        training_seq = []

        for depgraph in depgraphs:
            if not self._is_projective(depgraph):
                continue

            countProj += 1
            conf = modifiedConfiguration(depgraph)
            while len(conf.buffer) > 0:
                b0 = conf.buffer[0]
                features = conf.extract_features(include_features)
                binary_features = self._convert_to_binary_features(features)

                if len(conf.stack) > 0:
                    s0 = conf.stack[len(conf.stack) - 1]
                    # Left-arc operation
                    rel = self._get_dep_relation(b0, s0, depgraph)
                    if rel is not None:
                        key = Transition.LEFT_ARC + ':' + rel
                        self._write_to_file(key, binary_features, input_file)
                        operation.left_arc(conf, rel)
                        training_seq.append(key)
                        continue

                    # Right-arc operation
                    rel = self._get_dep_relation(s0, b0, depgraph)
                    if rel is not None:
                        key = Transition.RIGHT_ARC + ':' + rel
                        self._write_to_file(key, binary_features, input_file)
                        operation.right_arc(conf, rel)
                        training_seq.append(key)
                        continue

                    # reduce operation
                    flag = False
                    for k in range(s0):
                        if self._get_dep_relation(k, b0, depgraph) is not None:
                            flag = True
                        if self._get_dep_relation(b0, k, depgraph) is not None:
                            flag = True
                    if flag:
                        key = Transition.REDUCE
                        self._write_to_file(key, binary_features, input_file)
                        operation.reduce(conf)
                        training_seq.append(key)
                        continue

                # Shift operation as the default
                key = Transition.SHIFT
                self._write_to_file(key, binary_features, input_file)
                operation.shift(conf)
                training_seq.append(key)

        print(" Number of training examples : " + str(len(depgraphs)))
        print(" Number of valid (projective) examples : " + str(countProj))
        return training_seq
    
    def train(self, depgraphs, modelfile, include_features, modeltype, verbose=True):
        """
        :param depgraphs : list of DependencyGraph as the training data
        :type depgraphs : DependencyGraph
        :param modelfile : file name to save the trained model
        :type modelfile : str
        """

        try:
            input_file = tempfile.NamedTemporaryFile(
                prefix='transition_parse.train',
                dir=tempfile.gettempdir(),
                delete=False)

            if self._algorithm == self.ARC_STANDARD:
                self._create_training_examples_arc_std(depgraphs, input_file, include_features)
            else:
                self._create_training_examples_arc_eager(depgraphs, input_file, include_features)

            input_file.close()
            # Using the temporary file to train the libsvm classifier
            x_train, y_train = load_svmlight_file(input_file.name)
            # The parameter is set according to the paper:
            # Algorithms for Deterministic Incremental Dependency Parsing by Joakim Nivre
            # Todo : because of probability = True => very slow due to
            # cross-validation. Need to improve the speed here
            if modeltype=='mlp':
                model = MLPClassifier()
            elif modeltype == 'logistic_reg':
                model = LogisticRegression()
            elif modeltype == 'svm':
                model = svm.SVC(
                    kernel='poly',
                    degree=2,
                    coef0=0,
                    gamma=0.2,
                    C=0.5,
                    verbose=verbose,
                    probability=True)
                
            model.fit(x_train, y_train.ravel())
            # Save the model to file name (as pickle)
            pickle.dump(model, open(modelfile, 'wb'))
        finally:
            remove(input_file.name)
            
    def parse(self, depgraphs, modelFile, include_features):
        """
        :param depgraphs: the list of test sentence, each sentence is represented as a dependency graph where the 'head' information is dummy
        :type depgraphs: list(DependencyGraph)
        :param modelfile: the model file
        :type modelfile: str
        :return: list (DependencyGraph) with the 'head' and 'rel' information
        """
        result = []
        # First load the model
        model = pickle.load(open(modelFile, 'rb'))
        operation = Transition(self._algorithm)

        for depgraph in depgraphs:
            conf = modifiedConfiguration(depgraph)
            while len(conf.buffer) > 0:
                features = conf.extract_features(include_features)
                col = []
                row = []
                data = []
                for feature in features:
                    if feature in self._dictionary:
                        col.append(self._dictionary[feature])
                        row.append(0)
                        data.append(1.0)
                np_col = array(sorted(col))  # NB : index must be sorted
                np_row = array(row)
                np_data = array(data)

                x_test = sparse.csr_matrix((np_data, (np_row, np_col)), shape=(1, len(self._dictionary)))
                prob_dict = {}
                pred_prob = model.predict_proba(x_test)[0]
                for i in range(len(pred_prob)):
                    prob_dict[i] = pred_prob[i]
                sorted_Prob = sorted(
                    prob_dict.items(),
                    key=itemgetter(1),
                    reverse=True)

                # Note that SHIFT is always a valid operation
                for (y_pred_idx, confidence) in sorted_Prob:
                    #y_pred = model.predict(x_test)[0]
                    # From the prediction match to the operation
                    y_pred = model.classes_[y_pred_idx]

                    if y_pred in self._match_transition:
                        strTransition = self._match_transition[y_pred]
                        baseTransition = strTransition.split(":")[0]

                        if baseTransition == Transition.LEFT_ARC:
                            if operation.left_arc(conf, strTransition.split(":")[1]) != -1:
                                break
                        elif baseTransition == Transition.RIGHT_ARC:
                            if operation.right_arc(conf, strTransition.split(":")[1]) != -1:
                                break
                        elif baseTransition == Transition.REDUCE:
                            if operation.reduce(conf) != -1:
                                break
                        elif baseTransition == Transition.SHIFT:
                            if operation.shift(conf) != -1:
                                break
                    else:
                        raise ValueError("The predicted transition is not recognized, expected errors")

            # Finish with operations build the dependency graph from Conf.arcs

            new_depgraph = deepcopy(depgraph)
            for key in new_depgraph.nodes:
                node = new_depgraph.nodes[key]
                node['rel'] = ''
                # With the default, all the token depend on the Root
                node['head'] = 0
            for (head, rel, child) in conf.arcs:
                c_node = new_depgraph.nodes[child]
                c_node['head'] = head
                c_node['rel'] = rel
            result.append(new_depgraph)

        return result

In [0]:
def print_results(transition_type,de_model,model,string):
    
    if transition_type == 'arc-standard':
        tr = 'Arc Standard'
    else:
        tr = 'Arc Eager'
    
    if model == 'svm':
        m = 'SVC'
    elif model == 'logistic_reg':
        m = 'Logistic Regression'
    else:
        m = 'MLP'
    
    print(tr+' Transition | '+m+' | '+string)
    print('Labeled Attachment Score   : {:0.4f}'.format(de_model.eval()[0]))
    print('Unlabeled Attachment Score : {:0.4f}'.format(de_model.eval()[1]))
    

def transition_type(transition_type,train_dg,test_dg,model):
    
    if model == 'svm':
        print('----- SVM -----')
    elif model == 'logistic_reg':
        print('----- Logistic Regression -----')
    else:
        print('----- MLP -----')
    
    print('\n')
    print('With all features --- ')
    include_features = {'word': True,'lemma': True,'tag': True,'feats':True}
    parser_std = modifiedTransitionParser(transition_type)
    parser_std.train(train_dg,'arcstd1.model',include_features,model,verbose=False)

    result = parser_std.parse(test_dg ,'arcstd1.model', include_features)
    de_model = DependencyEvaluator(result, test_dg)
    print_results(transition_type,de_model,model,'All features')
    
    print('\n')
    print('Without morphological features ---')
    include_features = {'word': True,'lemma': True,'tag': True,'feats':False}
    parser_std = modifiedTransitionParser(transition_type)
    parser_std.train(train_dg,'arcstd2.model',include_features,model,verbose=False)

    result = parser_std.parse(test_dg ,'arcstd2.model', include_features)
    de_model = DependencyEvaluator(result, test_dg)
    print_results(transition_type,de_model,model,'w/o morphological features')
    
    print('\n')
    print('Without word features ---')
    include_features = {'word': False,'lemma': True,'tag': True,'feats':True}
    parser_std = modifiedTransitionParser(transition_type)
    parser_std.train(train_dg,'arcstd3.model',include_features,model,verbose=False)

    result = parser_std.parse(test_dg ,'arcstd3.model', include_features)
    de_model = DependencyEvaluator(result, test_dg)
    print_results(transition_type,de_model,model,'w/o word features')
    
    print('\n')
    print('Without lemma features ---')
    include_features = {'word': True,'lemma': False,'tag': True,'feats':True}
    parser_std = modifiedTransitionParser(transition_type)
    parser_std.train(train_dg,'arcstd4.model',include_features,model,verbose=False)

    result = parser_std.parse(test_dg ,'arcstd4.model', include_features)
    de_model = DependencyEvaluator(result, test_dg)
    print_results(transition_type,de_model,model,'w/o lemma features')
    
    print('\n')
    print('Without POS features')
    include_features = {'word': True,'lemma': True,'tag': False,'feats':True}
    parser_std = modifiedTransitionParser(transition_type)
    parser_std.train(train_dg,'arcstd5.model',include_features,model,verbose=False)

    result = parser_std.parse(test_dg ,'arcstd5.model', include_features)
    de_model = DependencyEvaluator(result, test_dg)
    print_results(transition_type,de_model,model,'w/o PoS features')


## Arc - Standard Transition

In [8]:
print('             ARC-STANDARD Transition          ')
print('\n')
print('\n')
print('\n')
print('Train file --- eng.train.conllu && Test file --- eng.test.conllu')
transition_type('arc-standard',train_graph_eng,test_graph_eng,'svm')
transition_type('arc-standard',train_graph_eng,test_graph_eng,'logistic_reg')
transition_type('arc-standard',train_graph_eng,test_graph_eng,'mlp')

print('\n')
print('\n')
print('Train file --- eng.train.conllu && Test file --- spanish.test.conllu')
transition_type('arc-standard',train_graph_eng,test_graph_sp,'svm')
transition_type('arc-standard',train_graph_eng,test_graph_sp,'logistic_reg')
transition_type('arc-standard',train_graph_eng,test_graph_sp,'mlp')

print('\n')
print('\n')
print('Train file --- spanish.train.conllu && Test file --- spanish.test.conllu')
transition_type('arc-standard',train_graph_sp,test_graph_sp,'svm')
transition_type('arc-standard',train_graph_sp,test_graph_sp,'logistic_reg')
transition_type('arc-standard',train_graph_sp,test_graph_sp,'mlp')

print('\n')
print('\n')
print('Train file --- spanish.train.conllu && Test file --- eng.test.conllu')
transition_type('arc-standard',train_graph_sp,test_graph_eng,'svm')
transition_type('arc-standard',train_graph_sp,test_graph_eng,'logistic_reg')
transition_type('arc-standard',train_graph_sp,test_graph_eng,'mlp')

             ARC-STANDARD Transition          






Train file --- eng.train.conllu && Test file --- eng.test.conllu
----- SVM -----


With all features --- 
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | SVC | All features
Labeled Attachment Score   : 0.8009
Unlabeled Attachment Score : 0.7153


Without morphological features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | SVC | w/o morphological features
Labeled Attachment Score   : 0.8204
Unlabeled Attachment Score : 0.7388


Without word features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | SVC | w/o word features
Labeled Attachment Score   : 0.8275
Unlabeled Attachment Score : 0.7432


Without lemma features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | SVC | w/o lemma features
Labele

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Arc Standard Transition | Logistic Regression | All features
Labeled Attachment Score   : 0.8013
Unlabeled Attachment Score : 0.7131


Without morphological features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | Logistic Regression | w/o morphological features
Labeled Attachment Score   : 0.7925
Unlabeled Attachment Score : 0.7047


Without word features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | Logistic Regression | w/o word features
Labeled Attachment Score   : 0.7929
Unlabeled Attachment Score : 0.7162


Without lemma features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | Logistic Regression | w/o lemma features
Labeled Attachment Score   : 0.7947
Unlabeled Attachment Score : 0.7086


Without POS features
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc St

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Arc Standard Transition | Logistic Regression | All features
Labeled Attachment Score   : 0.4324
Unlabeled Attachment Score : 0.2784


Without morphological features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | Logistic Regression | w/o morphological features
Labeled Attachment Score   : 0.1623
Unlabeled Attachment Score : 0.0101


Without word features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | Logistic Regression | w/o word features
Labeled Attachment Score   : 0.4563
Unlabeled Attachment Score : 0.2972


Without lemma features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Standard Transition | Logistic Regression | w/o lemma features
Labeled Attachment Score   : 0.4718
Unlabeled Attachment Score : 0.3059


Without POS features
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc St

...

Arc Standard Transition | Logistic Regression | All features
Labeled Attachment Score   : 0.2532
Unlabeled Attachment Score : 0.1246


Without morphological features ---
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Standard Transition | Logistic Regression | w/o morphological features
Labeled Attachment Score   : 0.3645
Unlabeled Attachment Score : 0.1450


Without word features ---
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Standard Transition | Logistic Regression | w/o word features
Labeled Attachment Score   : 0.2882
Unlabeled Attachment Score : 0.1428


Without lemma features ---
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Standard Transition | Logistic Regression | w/o lemma features
Labeled Attachment Score   : 0.2909
Unlabeled Attachment Score : 0.1339


Without POS features
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc St

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Arc Standard Transition | MLP | w/o lemma features
Labeled Attachment Score   : 0.2887
Unlabeled Attachment Score : 0.1401


Without POS features
 Number of training examples : 246
 Number of valid (projective) examples : 233


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Arc Standard Transition | MLP | w/o PoS features
Labeled Attachment Score   : 0.0732
Unlabeled Attachment Score : 0.0004


## Arc - Eager Transition

In [9]:
print('             ARC-EAGER Transition          ')
print('\n')
print('\n')
print('\n')
print('Train file --- eng.train.conllu && Test file --- eng.test.conllu')
transition_type('arc-eager',train_graph_eng,test_graph_eng,'svm')
transition_type('arc-eager',train_graph_eng,test_graph_eng,'logistic_reg')
transition_type('arc-eager',train_graph_eng,test_graph_eng,'mlp')

print('\n')
print('\n')
print('Train file --- eng.train.conllu && Test file --- spanish.test.conllu')
transition_type('arc-eager',train_graph_eng,test_graph_sp,'svm')
transition_type('arc-eager',train_graph_eng,test_graph_sp,'logistic_reg')
transition_type('arc-eager',train_graph_eng,test_graph_sp,'mlp')

print('\n')
print('\n')
print('Train file --- spanish.train.conllu && Test file --- spanish.test.conllu')
transition_type('arc-eager',train_graph_sp,test_graph_sp,'svm')
transition_type('arc-eager',train_graph_sp,test_graph_sp,'logistic_reg')
transition_type('arc-eager',train_graph_sp,test_graph_sp,'mlp')

print('\n')
print('\n')
print('Train file --- spanish.train.conllu && Test file --- eng.test.conllu')
transition_type('arc-eager',train_graph_sp,test_graph_eng,'svm')
transition_type('arc-eager',train_graph_sp,test_graph_eng,'logistic_reg')
transition_type('arc-eager',train_graph_sp,test_graph_eng,'mlp')

             ARC-EAGER Transition          






Train file --- eng.train.conllu && Test file --- eng.test.conllu
----- SVM -----


With all features --- 
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | SVC | All features
Labeled Attachment Score   : 0.8102
Unlabeled Attachment Score : 0.7339


Without morphological features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | SVC | w/o morphological features
Labeled Attachment Score   : 0.8173
Unlabeled Attachment Score : 0.7437


Without word features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | SVC | w/o word features
Labeled Attachment Score   : 0.8213
Unlabeled Attachment Score : 0.7424


Without lemma features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | SVC | w/o lemma features
Labeled Attachment Sc

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Arc Eager Transition | Logistic Regression | All features
Labeled Attachment Score   : 0.8102
Unlabeled Attachment Score : 0.7375


Without morphological features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | Logistic Regression | w/o morphological features
Labeled Attachment Score   : 0.8027
Unlabeled Attachment Score : 0.7282


Without word features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | Logistic Regression | w/o word features
Labeled Attachment Score   : 0.8062
Unlabeled Attachment Score : 0.7251


Without lemma features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | Logistic Regression | w/o lemma features
Labeled Attachment Score   : 0.7978
Unlabeled Attachment Score : 0.7251


Without POS features
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transiti

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Arc Eager Transition | Logistic Regression | All features
Labeled Attachment Score   : 0.1515
Unlabeled Attachment Score : 0.0701


Without morphological features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | Logistic Regression | w/o morphological features
Labeled Attachment Score   : 0.0813
Unlabeled Attachment Score : 0.0069


Without word features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | Logistic Regression | w/o word features
Labeled Attachment Score   : 0.0976
Unlabeled Attachment Score : 0.0286


Without lemma features ---
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transition | Logistic Regression | w/o lemma features
Labeled Attachment Score   : 0.1200
Unlabeled Attachment Score : 0.0528


Without POS features
 Number of training examples : 458
 Number of valid (projective) examples : 422
Arc Eager Transiti

...

Arc Eager Transition | Logistic Regression | All features
Labeled Attachment Score   : 0.7173
Unlabeled Attachment Score : 0.6670


Without morphological features ---
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Eager Transition | Logistic Regression | w/o morphological features
Labeled Attachment Score   : 0.6544
Unlabeled Attachment Score : 0.5741


Without word features ---
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Eager Transition | Logistic Regression | w/o word features
Labeled Attachment Score   : 0.6963
Unlabeled Attachment Score : 0.6482


Without lemma features ---
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Eager Transition | Logistic Regression | w/o lemma features
Labeled Attachment Score   : 0.6960
Unlabeled Attachment Score : 0.6457


Without POS features
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Eager Transiti

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Arc Eager Transition | Logistic Regression | All features
Labeled Attachment Score   : 0.2603
Unlabeled Attachment Score : 0.1211


Without morphological features ---
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Eager Transition | Logistic Regression | w/o morphological features
Labeled Attachment Score   : 0.2514
Unlabeled Attachment Score : 0.0980


Without word features ---
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Eager Transition | Logistic Regression | w/o word features
Labeled Attachment Score   : 0.2470
Unlabeled Attachment Score : 0.1304


Without lemma features ---
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Eager Transition | Logistic Regression | w/o lemma features
Labeled Attachment Score   : 0.2692
Unlabeled Attachment Score : 0.1149


Without POS features
 Number of training examples : 246
 Number of valid (projective) examples : 233
Arc Eager Transiti

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Arc Eager Transition | MLP | w/o PoS features
Labeled Attachment Score   : 0.0993
Unlabeled Attachment Score : 0.0146
